In [ ]:
using Gmsh: gmsh
using GridapGmsh
using Gridap
using Gridap.TensorValues
using PyPlot

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/3)*I4
I4_dev = I4_sym  - I4_vol

In [ ]:
const L = 25.5      #Length
const H = 19.1      #Height
const Lu = 4.6      #offset of load from left end
const Lh = 11.5     #notch length
const lsp = 0.08    #length factor
const CP = H/2      #center height  
const CH = 0.46     #Crack height
const hf = 0.06/2.1    #finer mesh size
const hfc = 0.06/50    #finer mesh size
const h = 60*hf     #coarser mesh size
const Lw = 2*h         #load width
const FMR = CH/2 

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
p1 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h)  
p2 = gmsh.model.geo.addPoint(Lu-0.5*Lw, 0.0, 0.0, h)
p3 = gmsh.model.geo.addPoint(Lu+0.5*Lw, 0.0, 0.0, h) 
p4 = gmsh.model.geo.addPoint(L, 0, 0.0, h) 
p5 = gmsh.model.geo.addPoint(L, H, 0.0, h)
p6 = gmsh.model.geo.addPoint(Lu+0.5*Lw, H, 0.0, h)
p7 = gmsh.model.geo.addPoint(Lu-0.5*Lw, H, 0.0, h)
p8 = gmsh.model.geo.addPoint(0, H, 0.0, h)
p9 = gmsh.model.geo.addPoint(0, CP + (CH*0.5), 0.0, h)
p10 = gmsh.model.geo.addPoint(Lh-(CH*0.5), CP + (CH*0.5), 0.0, h)
p11 = gmsh.model.geo.addPoint(Lh, CP, 0.0, hfc)
p12 = gmsh.model.geo.addPoint(Lh-(CH*0.5), CP - (CH*0.5), 0.0, h)
p13 = gmsh.model.geo.addPoint(0, CP - (CH*0.5), 0.0, h)

pp14 = gmsh.model.geo.addPoint(L, CP, 0.0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p6)
l6 = gmsh.model.geo.addLine(p6, p7)
l7 = gmsh.model.geo.addLine(p7, p8)
l8 = gmsh.model.geo.addLine(p8, p9)
l9 = gmsh.model.geo.addLine(p9, p10)
l10 = gmsh.model.geo.addLine(p10, p11)
l11 = gmsh.model.geo.addLine(p11, p12)
l12 = gmsh.model.geo.addLine(p12, p13)
l13 = gmsh.model.geo.addLine(p13, p1)

ll1 = gmsh.model.geo.addLine(p11, pp14)

cl1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13])

ps1 = gmsh.model.geo.addPlaneSurface([cl1])

pg1 = gmsh.model.addPhysicalGroup(2, [ps1])
pg2 = gmsh.model.addPhysicalGroup(1, [l2])
pg3 = gmsh.model.addPhysicalGroup(1, [l6])
pg4 = gmsh.model.addPhysicalGroup(1, [l7,l6,l5])
pg5 = gmsh.model.addPhysicalGroup(1, [l1,l2,l3])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, pg2, "DirichletBot")
gmsh.model.setPhysicalName(1, pg3, "DirichletTop")
gmsh.model.setPhysicalName(1, pg4, "ElectricPotentialTop")
gmsh.model.setPhysicalName(1, pg5, "ElectricPotentialBot")

gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", 0.95*Lh)
gmsh.model.mesh.field.setNumber(11, "XMax", L)
gmsh.model.mesh.field.setNumber(11, "YMin", CP-4*FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", CP+4*FMR)

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [ll1])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", 4*FMR)
gmsh.model.mesh.field.setNumber(2, "DistMax", 1)

gmsh.model.mesh.field.setAsBackgroundMesh(2)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("PlateWithEdgeNotch.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("PlateWithEdgeNotch.msh")
writevtk(model,"PlateWithEdgeNotch")

In [ ]:
ElecF = 5
phiMaxApp = (ElecF/10)*H*1e3
thickness = 5.1

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension)

In [ ]:
const Mat_tag = get_tag_from_name(labels,"Domain")

In [ ]:
const λ_mat = 77.8e3 
const μ_mat = 30.6e3

In [ ]:
const K₁₁_mat = 6e-9
const K₂₂_mat = 6e-9

In [ ]:
const Gc = 7e-3
const η = 1e-10

In [ ]:
p = VectorValue(0,1)
I = [1 0; 0 1]
K1 = zeros(2,2,2)
K2 = zeros(2,2,2)
K3 = zeros(2,2,2)
C1 = -6.98e-3
C2 = -6.06e-3
C3 = 16.00e-3
for i = 1:2, j = 1:2, k= 1:2
    K1[i,j,k] = p[i]*I[j,k]
end
for i = 1:2, j = 1:2, k= 1:2
    K2[i,j,k] = p[i]*p[j]*p[k]
end
for i = 1:2, j = 1:2, k= 1:2
    K3[i,j,k] = 0.5*(p[j]*I[i,k] + p[k]*I[i,j])
end
K = C1*K1+C2*K2+C3*K3
K[1,:,:]

In [ ]:
vals = zeros(2,2,2);
    vals[1,:,:] .= K[1,:,:]
    vals[2,:,:] .= K[2,:,:]
const e_mat = ThirdOrderTensorValue(vals ...)

In [ ]:
const K_mat = TensorValue(K₁₁_mat,0.0,0.0, K₁₁_mat)

In [ ]:
σ_elas(ε) = λ_mat*tr(ε)*one(ε) + 2*μ_mat*ε

function σ_elasMod(ε, ε_in, s_in)  
 if tr(ε_in)  >= 0
      σ = (s_in^2 + η)*σ_elas(ε)
  elseif tr(ε_in) < 0
      σ = (s_in^2 + η)*I4_dev ⊙ σ_elas(ε) + I4_vol⊙ σ_elas(ε)
  end  
    return σ
end

In [ ]:
σ_piezo(∇) = ∇ ⋅ e_mat

σ_piezoMod(∇, ∇_in, s_in) = (s_in^2 + η)*σ_piezo(∇)

In [ ]:
D_elasMod(ε, ∇_in, s_in) = (s_in^2 + η)*(e_mat ⋅² ε)

In [ ]:
D_piezo(∇) = K_mat ⋅ ∇

function D_piezoMod(∇, s_in)   
    return (s_in^2 + η)*D_piezo(∇)
end

In [ ]:
function ψPos(ε_in, ∇_in)   
 if tr(ε_in) >= 0
    ψPlus = 0.5*(ε_in ⊙ σ_elas(ε_in)) + 0.5*(σ_piezo(∇_in) ⊙ ε_in)
 elseif tr(ε_in) < 0
    ψPlus = 0.5*((I4_dev ⊙ σ_elas(ε_in)) ⊙ (I4_dev ⊙ ε_in)) + 0.5*(σ_piezo(∇_in) ⊙ (I4_dev ⊙ ε_in))        
 end 
    return ψPlus
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_in = ψhPos_in
  if ψPlus_in >= ψPlusPrev_in
    ψPlus_out = ψPlus_in
  else
    ψPlus_out = ψPlusPrev_in
  end
  true,ψPlus_out
end

## FE formulation

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
order = 1
degree = 2*order

In [ ]:
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
LoadTagId = get_tag_from_name(labels,"DirichletTop")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model,reffe_PF;
  conformity=:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletTop", "DirichletBot"],
          dirichlet_masks=[(false,true), (false,true)])

uh = zero(V0_Disp)

In [ ]:
reffe_ElecPot = ReferenceFE(lagrangian,Float64,order)
V0_ElecPot  = TestFESpace(model,reffe_ElecPot;
  conformity=:H1,
  dirichlet_tags=["ElectricPotentialTop","ElectricPotentialBot"])

In [ ]:
V0 = MultiFieldFESpace([V0_Disp,V0_ElecPot])

In [ ]:
function  stepPhaseField(uh_in,ψPlusPrev_in)
        
        a_PF(s,ϕ) = ∫( Gc*lsp*∇(ϕ)⋅ ∇(s) + 2*ψPlusPrev_in*s*ϕ  + (Gc/lsp)*s*ϕ )*dΩ
        b_PF(ϕ) = ∫( (Gc/lsp)*ϕ )*dΩ
        op_PF = AffineFEOperator(a_PF,b_PF,U_PF,V0_PF)
        sh_out = solve(op_PF)           
    
    return sh_out
    
end

In [ ]:
 function   stepDispElecPot(uh_in,phih_in,sh_in,vApp,phiApp)
    
        uApp1(x) = VectorValue(0.0,vApp)
        uApp2(x) = VectorValue(0.0,-vApp)
        U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])
    
        phiApp1(x) = 0
        phiApp2(x) = phiApp
        U_ElecPot = TrialFESpace(V0_ElecPot,[phiApp1,phiApp2])
    
        U = MultiFieldFESpace([U_Disp,U_ElecPot])
    
        a((u,ϕ),(v,ψ)) = ∫( (ε(v) ⊙ (σ_elasMod∘(ε(u),ε(uh_in),sh_in))) + (∇(v) ⊙ (σ_piezoMod ∘(∇(ϕ),∇(phih_in),sh_in))) - (∇(ψ)⋅(D_piezoMod∘ (∇(ϕ),sh_in))) + (∇(ψ)⋅(D_elasMod∘(ε(u),∇(phih_in),sh_in))) )*dΩ
        b((v,ψ)) = 0.0
    
        op = AffineFEOperator(a,b,U,V0)
        uhPhi = solve(op)
        uh_out,phih_out = uhPhi
    
    return uh_out,phih_out
end

In [ ]:
vApp = 0
phiApp = 0
delv = 1e-4
vAppMax = 0.010
innerMax = 10
count = 0

Load = Float64[]
Displacement = Float64[]

ψPlusPrev = CellState(0.0,dΩ) 
sPrev = CellState(1.0,dΩ)
sh = project(sPrev,model,dΩ,order)
ϕPrev = CellState(0.0,dΩ)
ϕh = project(ϕPrev,model,dΩ,order)


while vApp .< vAppMax 
    count = count .+ 1
    vApp = vApp .+ delv
    
    if vApp <= 5e-4
        phiApp = vApp * (phiMaxApp/5e-4)
    else
        phiApp = phiMaxApp
    end
    
    if vApp >= 5e-4 && vApp <= 2e-3
        delv = 1e-4
    elseif vApp >= 2e-3
        delv = 1e-4
    end
    
    print("\n Entering displacemtent step$count :", float(vApp))
    
   for inner = 1:innerMax   
        
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
        
        RelErr = abs(sum(∫( Gc*lsp*∇(sh)⋅ ∇(sh) + 2*ψhPlusPrev*sh*sh  + (Gc/lsp)*sh*sh)*dΩ - ∫( (Gc/lsp)*sh)*dΩ))/abs(sum(∫( (Gc/lsp)*sh)*dΩ))
        print("\n Relative error = ",float(RelErr))
        
        sh = stepPhaseField(uh,ψhPlusPrev) 
        uh,ϕh = stepDispElecPot(uh,ϕh,sh,vApp,phiApp)
        
        ψhPos_in = ψPos∘(ε(uh),∇(ϕh))      
        
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in)
  
        if RelErr < 1e-8
            break 
        end      
    end
    
    Node_Force = sum(∫( n_Γ_Load ⋅ (σ_elasMod∘(ε(uh),ε(uh),sh)) ) *dΓ_Load + ∫( n_Γ_Load ⋅ (σ_piezoMod∘(∇(ϕh),∇(ϕh),sh) ) )  *dΓ_Load)
    
    push!(Load, thickness*Node_Force[2])
    push!(Displacement, vApp)
    if mod(count,2) == 0
         writevtk(Ω,"results_NotchedPlate$count",cellfields=
        ["uh"=>uh,"s"=>sh ,"phi"=>ϕh, "epsi"=>ε(uh)])
    end
end

In [ ]:
 writevtk(Ω,"results_NotchedPlate$count",cellfields=
        ["uh"=>uh,"s"=>sh ,"phi"=>ϕh, "epsi"=>ε(uh)])

In [ ]:
plot(Displacement,Load)